<a href="https://colab.research.google.com/github/ayyucedemirbas/machine_learning_algorithms/blob/master/k_fold_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
(input_train, target_train), (input_test, target_test) = cifar10.load_data()
# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

In [10]:
input_train.shape[1:]

(32, 32, 3)

In [15]:
inputs[1].shape

(32, 32, 3)

In [16]:
targets[1].shape

(1,)

In [12]:
import tensorflow
from tensorflow.keras import initializers

In [14]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=2, shuffle=False)
input_shape = input_train.shape[1:]
# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold=[]
loss_per_fold=[]
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model= Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3),kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu',kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(256, activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        Dense(128, activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        Dense(10, activation='softmax', kernel_initializer=initializers.GlorotUniform(seed=None))


      ])

  # Compile the model
  opt = tensorflow.keras.optimizers.Adam(learning_rate=0.005)
  acc = tensorflow.keras.metrics.SparseCategoricalAccuracy()
  mae = tensorflow.keras.metrics.MeanAbsoluteError()
  model.compile(
    
      optimizer= opt,
      loss = 'sparse_categorical_crossentropy', #sparse_categorical_crossentropy --> data labels are integers
      metrics= [acc, mae]
  )


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],epochs=8, batch_size=256, verbose=2)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/8
118/118 - 35s - loss: 1.9185 - sparse_categorical_accuracy: 0.2786 - mean_absolute_error: 4.4216
Epoch 2/8
118/118 - 33s - loss: 1.4638 - sparse_categorical_accuracy: 0.4625 - mean_absolute_error: 4.4216
Epoch 3/8
118/118 - 33s - loss: 1.2752 - sparse_categorical_accuracy: 0.5387 - mean_absolute_error: 4.4216
Epoch 4/8
118/118 - 32s - loss: 1.1612 - sparse_categorical_accuracy: 0.5862 - mean_absolute_error: 4.4216
Epoch 5/8
118/118 - 33s - loss: 1.0782 - sparse_categorical_accuracy: 0.6165 - mean_absolute_error: 4.4216
Epoch 6/8
118/118 - 33s - loss: 0.9920 - sparse_categorical_accuracy: 0.6468 - mean_absolute_error: 4.4216
Epoch 7/8
118/118 - 33s - loss: 0.9523 - sparse_categorical_accuracy: 0.6627 - mean_absolute_error: 4.4216
Epoch 8/8
118/118 - 32s - loss: 0.8822 - sparse_categorical_accuracy: 0.6882 - mean_absolute_error: 4.4216
Score for fold 1: loss of 1.071470022201538; sp